In [1]:
import itertools
from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision

In [2]:
trn_dataset = datasets.MNIST('./mnist_data/',
                             download=False,
                             train=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(), # image to Tensor
                                 transforms.Normalize((0.1307,), (0.3081,)) # image, label
                             ]))

val_dataset = datasets.MNIST("./mnist_data/",
                             download=False,
                             train=False,
                             transform= transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307, ),(0.3081, ))
                           ]))

In [3]:
batch_size = 64
trn_loader = torch.utils.data.DataLoader(trn_dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                        drop_last=True)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                        drop_last=True)

In [4]:
# construct model on cuda if available
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cpu


In [5]:
class CNNClassifier(nn.Module):

    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNNClassifier, self).__init__()
        conv1 = nn.Conv2d(1, 6, 5, 1)  # 6@24*24
        # activation ReLU
        pool1 = nn.MaxPool2d(2)  # 6@12*12
        conv2 = nn.Conv2d(6, 16, 5, 1)  # 16@8*8
        # activation ReLU
        pool2 = nn.MaxPool2d(2)  # 16@4*4

        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2
        )

        fc1 = nn.Linear(16 * 4 * 4, 120)
        # activation ReLU
        fc2 = nn.Linear(120, 84)
        # activation ReLU
        fc3 = nn.Linear(84, 10)

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.ReLU(),
            fc3
        )

        # gpu로 할당
        if use_cuda:
            self.conv_module = self.conv_module.cuda()
            self.fc_module = self.fc_module.cuda()

    def forward(self, x):
        out = self.conv_module(x)  # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]:  # 16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return F.softmax(out, dim=1)

cnn = CNNClassifier()

In [6]:
# loss
criterion = nn.CrossEntropyLoss()
# backpropagation method
learning_rate = 1e-3
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
# hyper-parameters
num_epochs = 2
num_batches = len(trn_loader)

trn_loss_list = []
val_loss_list = []
for epoch in range(num_epochs):
    trn_loss = 0.0
    for i, data in enumerate(trn_loader):
        x, label = data
        if use_cuda:
            x = x.cuda()
            label = label.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        model_output = cnn(x)
        # calculate loss
        loss = criterion(model_output, label)
        # back propagation
        loss.backward()
        # weight update
        optimizer.step()

        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del model_output

        # 학습과정 출력
        if (i + 1) % 100 == 0:  # every 100 mini-batches
            with torch.no_grad():  # very very very very important!!!
                val_loss = 0.0
                for j, val in enumerate(val_loader):
                    val_x, val_label = val
                    if use_cuda:
                        val_x = val_x.cuda()
                        val_label = val_label.cuda()
                    val_output = cnn(val_x)
                    v_loss = criterion(val_output, val_label)
                    val_loss += v_loss

            print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, num_batches, trn_loss / 100, val_loss / len(val_loader)
            ))

            trn_loss_list.append(trn_loss / 100)
            val_loss_list.append(val_loss / len(val_loader))
            trn_loss = 0.0

epoch: 1/2 | step: 100/937 | trn loss: 1.9557 | val loss: 1.7077
epoch: 1/2 | step: 200/937 | trn loss: 1.6718 | val loss: 1.6285
epoch: 1/2 | step: 300/937 | trn loss: 1.6259 | val loss: 1.5721
epoch: 1/2 | step: 400/937 | trn loss: 1.5610 | val loss: 1.5335
epoch: 1/2 | step: 500/937 | trn loss: 1.5325 | val loss: 1.5341
epoch: 1/2 | step: 600/937 | trn loss: 1.5187 | val loss: 1.5210
epoch: 1/2 | step: 700/937 | trn loss: 1.5142 | val loss: 1.5113
epoch: 1/2 | step: 800/937 | trn loss: 1.5127 | val loss: 1.5038
epoch: 1/2 | step: 900/937 | trn loss: 1.5059 | val loss: 1.4984
epoch: 2/2 | step: 100/937 | trn loss: 1.5001 | val loss: 1.4917
epoch: 2/2 | step: 200/937 | trn loss: 1.4997 | val loss: 1.4917
epoch: 2/2 | step: 300/937 | trn loss: 1.4982 | val loss: 1.4914
epoch: 2/2 | step: 400/937 | trn loss: 1.4950 | val loss: 1.4873
epoch: 2/2 | step: 500/937 | trn loss: 1.4908 | val loss: 1.4957
epoch: 2/2 | step: 600/937 | trn loss: 1.4932 | val loss: 1.4961
epoch: 2/2 | step: 700/93

In [ ]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
#     X_test, Y_test = val_loader
    for i, data in enumerate(val_loader):
        x_test, Y_test = data

        prediction = cnn(x_test)
        print(prediction)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

In [ ]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.

    # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
    r = random.randint(0, len(val_loader) - 1)
    X_single_data = val_loader[r:r + 1][0].view(-1, 28 * 28).float().to(device)
    Y_single_data = val_loader[r:r + 1][1]

    print('Label: ', Y_single_data.item())
    single_prediction = cnn(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(x_test[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()

In [37]:
trans = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                            transforms.Resize((28,28)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307, ),(0.3081, ))])

data = torchvision.datasets.ImageFolder(root = './data',
                                       transform = trans)

In [38]:
loader = DataLoader(data,
                   shuffle = False)

dataiter = iter(loader)
images, labels = dataiter.next()
print(labels)

tensor([0])


In [39]:
# prediction = cnn(images)
# print(prediction)

for data in loader:
    x_test, Y_test = data
    
    print('Label: ', Y_test.item())
    single_prediction = cnn(x_test)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Label:  0
Prediction:  8
Label:  0
Prediction:  8
Label:  0
Prediction:  0
Label:  1
Prediction:  8
Label:  1
Prediction:  8
Label:  1
Prediction:  0
Label:  2
Prediction:  2
Label:  2
Prediction:  2
Label:  2
Prediction:  8
Label:  3
Prediction:  8
Label:  3
Prediction:  2
Label:  3
Prediction:  8
Label:  4
Prediction:  5
Label:  4
Prediction:  3
Label:  4
Prediction:  0
Label:  5
Prediction:  5
Label:  5
Prediction:  8
Label:  5
Prediction:  5
Label:  6
Prediction:  5
Label:  6
Prediction:  8
Label:  6
Prediction:  0
Label:  7
Prediction:  0
Label:  7
Prediction:  2
Label:  7
Prediction:  8
Label:  8
Prediction:  8
Label:  8
Prediction:  2
Label:  8
Prediction:  5
Label:  9
Prediction:  0
Label:  9
Prediction:  3
Label:  9
Prediction:  0
